In [1]:
from six.moves import cPickle as pickle
import numpy as np
import os
from scipy.misc import imread

In [2]:
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='latin1')
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
    return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)    
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [3]:
[Xtr, Ytr, Xte, Yte] = load_CIFAR10('../cifar-10-batches-py/')

In [4]:
Xtr

array([[[[  59.,   62.,   63.],
         [  43.,   46.,   45.],
         [  50.,   48.,   43.],
         ..., 
         [ 158.,  132.,  108.],
         [ 152.,  125.,  102.],
         [ 148.,  124.,  103.]],

        [[  16.,   20.,   20.],
         [   0.,    0.,    0.],
         [  18.,    8.,    0.],
         ..., 
         [ 123.,   88.,   55.],
         [ 119.,   83.,   50.],
         [ 122.,   87.,   57.]],

        [[  25.,   24.,   21.],
         [  16.,    7.,    0.],
         [  49.,   27.,    8.],
         ..., 
         [ 118.,   84.,   50.],
         [ 120.,   84.,   50.],
         [ 109.,   73.,   42.]],

        ..., 
        [[ 208.,  170.,   96.],
         [ 201.,  153.,   34.],
         [ 198.,  161.,   26.],
         ..., 
         [ 160.,  133.,   70.],
         [  56.,   31.,    7.],
         [  53.,   34.,   20.]],

        [[ 180.,  139.,   96.],
         [ 173.,  123.,   42.],
         [ 186.,  144.,   30.],
         ..., 
         [ 184.,  148.,   94.],
       

In [5]:
print(Ytr)

[6 9 9 ..., 9 1 1]


In [6]:
from sklearn import svm

clf = svm.SVC()
clf.fit(Xtr, Ytr)

ValueError: Found array with dim 4. Estimator expected <= 2.

In [ ]:
clf.predict(Xte)